# Cloud Data Pipeline for International Bank for Reconstruction and Development(IBRD)
### Data Engineering Capstone Project

#### Project Summary
The International Bank for Reconstruction and Development, wants to move their processes and data pipeline onto the cloud. Their data resides in S3, in a directory of CSV on the transcation history of the loans. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Clean and Save the Data
* Step 4: Define the Data Model
* Step 5: Run ETL to Model the Data
* Step 6: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
* Project Description

In this project, we will build an ETL pipeline that extracts data from S3, processes and clean them using Spark, and loads the data back into S3 as a set of dimensional tables in csv format. Then, create cluster with redshift and load data into redshift. This will allow our analytics team to continue finding insights of our loan users.

* Tool

 In this porject, we will use AWS S3, Redshift, Spark, Python
 
#### Describe and Gather Data 
The data is coming from World Bank and you can download it from below page:

https://finances.worldbank.org/Loans-and-Credits/IBRD-Statement-Of-Loans-Historical-Data/zucq-nrc3
It is around 300mb and below is the data layout of the file.

##### End of PeriodEnd of Period Date 
represents the date as of which balances are shown in the report.
##### Loan Number
For IBRD loans and IDA credits or grants a loan number consists of the organization prefix (IBRD/IDA) and a five-character label that uniquely identifies the loan within the organization. In IDA, all grant labels start with the letter ‘H’.
#####  RegionCountry 
lending is grouped into regions based on the current World Bank administrative (rather than geographic) region where project implementation takes place. The Other Region is used for loans to the IFC.
##### Country Code
Country Code according to the World Bank country list. Might be different from the ISO country code.
##### Country
Country to which loan has been issued. Loans to the IFC are included under the country “World”.
##### Borrower
The representative of the borrower to which the Bank loan is made.
##### Guarantor Country 
CodeCountry Code of the Guarantor according to the World Bank country list. Might be different from the ISO country code.
##### Guarantor
The Guarantor guarantees repayment to the Bank if the borrower does not repay.
##### Loan 
TypeA type of loan/loan instrument for which distinctive accounting and/or other actions need to be performed. See Data Dictionary attached in the About section or Data Dictionary dataset available from the list of all datasets for details.
##### Loan StatusStatus of the loan. 
See Data Dictionary attached in the About section or Data Dictionary dataset available from the list of all datasets for status descriptions.
##### Interest Rate
Current Interest rate or service charge applied to loan. For loans that could have more than one interest rate (e.g. FSL or SCL fixed rate loans), the interest rate is shown as “0”.
##### Currency of Commitment
The currency in which a borrower’s loan, credit, grant or trust fund is denominated.
##### Project 
IDA Bank project is referenced by a project ID (Pxxxxxxx). More than one loan or credit may be associated with one Project ID.
##### Project Name 
Short descriptive project name.
##### Original Principal Amount
The original US dollar amount of the loan that is committed and approved.
##### Cancelled Amount
The portion of the undisbursed balance which has been cancelled (i.e. no longer available for future disbursement). Cancellations include terminations (where approved loan agreements were never signed).
##### Undisbursed Amount
The amount of a loan commitment that is still available to be drawn down. These currency amounts have been converted to US dollars at the exchange rates applicable at the end of period date.
##### Disbursed Amount
The amount that has been disbursed from a loan commitment in equivalent US dollars, calculated at the exchange rate on the value date of the individual disbursements.
##### Repaid to IBRD
Total principal amounts paid or prepaid to IBRD in US dollars, calculated at the exchange rate on the value date of the individual repayments.
##### Due to IBRD
Where the exchange adjustment is shown separately, this is the amount disbursed and outstanding expressed as a stock of debt in historical US Dollars. Where the exchange adjustment is not shown separately, this is the amount due and outstanding as of th
##### Exchange Adjustment
The increase (decrease) in value of disbursed and outstanding amount due to exchange rate fluctuations. This amount added to “Due to IBRD” yields “Borrower’s Obligation; includes exchange adjustments on the amounts Due to 3rd parties.
##### Borrower's Obligation
The Borrower Obligation is the outstanding balance for the loan as of the end of period date in US dollars equivalent. The Borrower's Obligation includes the amounts outstanding Due to 3rd parties.
##### Sold 3rd Party
Portion of loan sold to a third party.
Repaid 3rd PartyAmount repaid to a third party.
##### Due 3rd Party
Amount due to a third party.
Loans HeldThe sum of the disbursed and outstanding amounts (net of repayments, i.e. Due to IBRD/IDA) plus undisbursed available amounts expressed in historical US Dollars.
##### First Repayment Date
The date on which principal repayment starts.
##### Last Repayment Date
The date specified in the loan/credit agreement (amended for any partial prepayments) on which the last principal installment must be repaid by the Borrower.
##### Agreement Signing Date
The date the borrower and the Bank sign the loan agreement.
##### Board Approval Date
The date the World Bank approves the loan.
##### Effective Date (Most Recent)
The date on which a legal agreement becomes effective, or is expected to become effective.
##### Closed Date (Most Recent)
The date specified in the legal agreement (or extension) after which the Bank may, by notice to the borrower, terminate the right to make withdrawals from the loan account.
##### Last Disbursement Date
The date on which the last disbursement was made (prior to the end of period date).

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


In [3]:
# Import all modules and setup here
# Loading all library
import numpy as np
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import boto3
import time
from functools import reduce
import pandas as pd
# Change padans parameter to adjust visliazation
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', 200)
# Load in aws credential
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['KEY']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['SECRET']

In [4]:
# Define a function that creates spark session
def create_spark_session():
    """
    This function is used to create a spark session to work in
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
# Create spark session
spark = create_spark_session()

In [5]:
# Load data from AWS s3
df_spark = spark.read.csv("s3a://udacity-leejohn/loan/ibrd-statement-of-loans-historical-data.csv", header=True)
# Function that will uppercase everything in the dataframe
fields = df_spark.schema.fields
stringFields = filter(lambda f: isinstance(f.dataType, StringType), fields)
nonStringFields = map(lambda f: col(f.name), filter(lambda f: not isinstance(f.dataType, StringType), fields))
stringFieldsTransformed = map(lambda f: upper(col(f.name)), stringFields) 
allFields = [*stringFieldsTransformed, *nonStringFields]
df_new = df_spark.select(allFields)
# Rename the column name
# Get old column names 
oldColumns = df_new.schema.names
# Setup new column names
newColumns  = ['End_of_Period', 'Loan_Number', 'Region', 'Country_Code', 'Country','Borrower','Guarantor_Country_Code','Guarantor','Loan_Type','Loan_Status','Interest_Rate','Currency_of_Commitment','Project_ID','Project_Name','Original_Principal_Amount','Cancelled_Amount','Undisbursed_Amount','Disbursed_Amount','Repaid_to_IBRD','Due_to_IBRD','Exchange_Adjustment','Borrowers_Obligation','Sold_3rd_Party','Repaid_3rd_Party','Due_3rd_Party','Loans_Held','First_Repayment_Date','Last_Repayment_Date','Agreement_Signing_Date','Board_Approval_Date','Effective_Date_Most_Recent','Closed_Date_Most_Recent','Last_Disbursement_Date']
# Rename the dataframe
df = reduce(lambda df_spark, idx: df_spark.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_new)

In [ ]:
# Count the number of rows in data
df.count()

In [7]:
# Describe the data
df.describe().toPandas()

,summary,End_of_Period,Loan_Number,Region,Country_Code,Country,Borrower,Guarantor_Country_Code,Guarantor,Loan_Type,Loan_Status,Interest_Rate,Currency_of_Commitment,Project_ID,Project_Name,Original_Principal_Amount,Cancelled_Amount,Undisbursed_Amount,Disbursed_Amount,Repaid_to_IBRD,Due_to_IBRD,Exchange_Adjustment,Borrowers_Obligation,Sold_3rd_Party,Repaid_3rd_Party,Due_3rd_Party,Loans_Held,First_Repayment_Date,Last_Repayment_Date,Agreement_Signing_Date,Board_Approval_Date,Effective_Date_Most_Recent,Closed_Date_Most_Recent,Last_Disbursement_Date
0,count,802137,802137,802137,802137,802137,796580,770595,743002,802089,802089,775755,48,770748,603238,802089,802089,802089,802089,802089,802089,802089,802089,802089,802089,802089,802089,798315,798411,792062,802041,797438,801295,450242
1,mean,None,None,None,None,None,0.6070833333333335,None,None,None,8000000.0,235.78813540232693,0.0,3636810.0200000014,4906399.120624998,7.683324418806152E7,9668568.033589192,7655266.693610078,5.9512477889023624E7,3.98461426406178E7,1.935132263823048E7,-214132.04400157375,1.9138333455590654E7,445558.31082740123,445558.31082740123,0.0,2.7008203155569147E7,None,None,None,None,None,None,None
2,stddev,None,None,None,None,None,0.40571428125490533,None,None,None,0.0,29374.007637237624,0.0,0.0,106784.82174868968,1.4862488000733185E8,4.464619032854254E7,5.437482376824411E7,1.2757727483751363E8,8.82332276180936E7,9.361462870492262E7,9208274.311718144,9.111660210168463E7,3881816.4242969775,3881816.4242969775,0.0,1.1411908294752274E8,None,None,None,None,None,None,None
3,min,2011-04-30T00:00:00.000,GFMDR,AFRICA,1W,REPAID,#MULTIVALUE,1W,ALBANIA,\tIFC LOAN,APPROVED,0,0,3636810.02,COAL IAP,0,0,-0.01,0,0,-0.01,-0.01,-0.01,0,0,0,-0.01,1930-02-15T00:00:00.000,1930-01-15T00:00:00.000,1947-05-09T00:00:00.000,1947-05-09T00:00:00.000,1947-06-09T00:00:00.000,1947-12-31T00:00:00.000,1983-10-11T00:00:00.000
4,max,"KA BANKA D.D.""",IBRDS0200,SOUTH ASIA,ZW,ZIMBABWE,ZONA FRANCA INDUST. Y COMERC.,ZW,ZIMBABWE,SNGL CRNCY,TERMINATED,9.97,0,P999186,ZOUXIAN THERMAL POWE,99963871.4,9999937.5,9999937.5,99999972.31,99999999.99,99990000,9998114.98,999975000,997000,997000,1997-11-18T00:00:00.000,9999515.72,2049-07-15T00:00:00.000,2061-08-15T00:00:00.000,2019-04-11T00:00:00.000,2019-04-30T00:00:00.000,2020-08-01T00:00:00.000,2033-12-31T00:00:00.000,2019-05-03T00:00:00.000


In [ ]:
# This is to display the number of null value of each column, before we do any cleaning, you will see null data count for each column with below command
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()

### Step 3: Cleaning Steps

In this step, we will clean data to remove any null value with different methods. 
This process will take about 8 min.

In [8]:
# Since below 2 columns have too much null values(average 50% null values) and are not necessary, so we drop them.  
df = df.drop('Last_Disbursement_Date')
df = df.drop('Currency_of_Commitment')
# For these 2 columns, if we look into the raw data,we will see that there are some "?", "@" in the data, which are some errors come from raw data. 
# I think this is the error comes from different coding type.
# Instead of trying to fix them, we can just drop them.
df = df.drop('Borrower')
df = df.drop('Project_Name')
start_time = time.time()

In [9]:
# Edit the loan_number to make it 9 digits
# From data layout, we can know that loan_number should follow a 9 dights pattern.
# Find records that have less than 9 digits Loan_Number, replace them to the correct format
df_6 = df.where(length(col("Loan_Number")) == 6).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{2})" , "$1000$2" ))
df_7 = df.where(length(col("Loan_Number")) == 7).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{3})" , "$100$2" ))
df_8 = df.where(length(col("Loan_Number")) == 8).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{4})" , "$10$2" ))

# No records with loan_number that has less than 9 digits
df = df.filter(length(col("Loan_Number")) == 9).union(df_6).union(df_7).union(df_8)

In [10]:
# If both Guarantor_Country_Code and Guarantor are empty, then it's hard to say whether they are suppose to be empty(no guarantor)
# or they are missing values, so I just drop them.
df = df.filter('Guarantor_Country_Code is not NULL or Guarantor is not NULL')

In [11]:
# For each loan number, there should be one country code. Run below code, we will find there are 3 records that have the wrong country data, so we drop them. 
#df.select('Loan_Number','Country_Code').distinct().groupBy('Loan_Number').count().withColumnRenamed('count', 'ccount').filter('count>1').toPandas()
#df.where(df.Loan_Number == 'IBRD82610').select('Country').groupBy('Country').count().toPandas()
#df.where(df.Loan_Number == 'IBRD82550').select('Country').groupBy('Country').count().toPandas()
#df.where(df.Loan_Number == 'IBRD82580').select('Country').groupBy('Country').count().toPandas()
df = df.where((df.Loan_Number == 'IBRD82580') & (df.Country != 'CHINA') | (df.Loan_Number != 'IBRD82580'))
df = df.where((df.Loan_Number == 'IBRD82550') & (df.Country != 'CHINA') | (df.Loan_Number != 'IBRD82550'))
df = df.where((df.Loan_Number == 'IBRD82610') & (df.Country != 'INDIA') | (df.Loan_Number != 'IBRD82610'))

In [12]:
# Remove all missing value for Project_ID column
# Create a dict that key is loan_number and value is project_id
x1 = df.filter('Project_ID is not NULL').select('Loan_Number','Project_ID').distinct().toPandas().set_index('Loan_Number')['Project_ID'].to_dict() 
# Create a pandas dataframe that only has the records that have missing project_id
pdf = df.filter('Project_ID is NULL').toPandas()
# Loop through the dataframe and replace its missing value with the loan_number
for index,row in pdf.iterrows():
    att = row.Loan_Number
    if att in x1.keys():
        row.Project_ID = x1.get(att)
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf.astype(str)).filter('Project_ID is not NULL').filter('Project_ID != \'None\'')
# Union it with the good records and get new dataframe
df = df.filter('Project_ID is not NULL').union(ddd)

In [13]:
# Remove all missing value for Guarantor and Guarantor_Country_Code column
# Create a dict that key is loan_number and value is a array of country, country_code
c1 = df.select('Loan_Number','Country','Country_Code').distinct().toPandas().set_index('Loan_Number').T.to_dict('list')
# By exploring data, we find there is one more country in Guarantor, which is united kingdom, we will add it to our dict
c1b = df.filter('Guarantor == \'UNITED KINGDOM\'').select('Loan_Number').distinct().collect()
for i in c1b:
    c1[i.Loan_Number] = ['UNITED KINGDOM','GB']
# Loop through the dataframe and replace its missing value with the loan_number
pdf = df.filter('Guarantor is NULL or Guarantor_Country_Code is NULL').toPandas()
for index,row in pdf.iterrows():
    att = row.Loan_Number
    if att in c1.keys():
        row.Guarantor = c1.get(att)[0]
        row.Guarantor_Country_Code = c1.get(att)[1]
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf).filter('Guarantor is not NULL and Guarantor_Country_Code is not NULL')
# Union it with the good records and get new dataframe
df = df.filter('Guarantor is not NULL and Guarantor_Country_Code is not NULL').union(ddd)

In [14]:
# Remove all missing value for Interest_Rate
# Since for each loan number, we will find different interest rate. So to handle this case, for rach loan number, I will use mean value of all intertest rate to repalce its nul value. 
# Create a dict that key is loan_number and value is intertest rate
i1 = df.filter('Interest_Rate is not NULL').filter('Interest_Rate != \'None\'').select('Loan_Number','Interest_Rate').distinct().toPandas()
i1.Interest_Rate = i1.Interest_Rate.astype(np.float16)
i1 = i1.groupby('Loan_Number', as_index=True).agg({"Interest_Rate": "mean"})['Interest_Rate'].to_dict()
pdf = df.filter('Interest_Rate is NULL').toPandas()
# Loop through the dataframe and replace its missing value with the loan_number
for index,row in pdf.iterrows():
    att = row.Interest_Rate
    if att in c1.keys():
        row.Interest_Rate = c1.get(att)
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf.astype(str)).filter('Interest_Rate is not NULL').filter('Interest_Rate != \'None\'')
# Union it with the good records and get new dataframe
df = df.filter('Interest_Rate is not NULL').union(ddd)
print("--- %s seconds ---" % (time.time() - start_time))

--- 463.7417585849762 seconds ---


In [16]:
# This is to display the number of null value of each column, before we do any cleaning, you will see null data count for each column with below command
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()

,End_of_Period,Loan_Number,Region,Country_Code,Country,Guarantor_Country_Code,Guarantor,Loan_Type,Loan_Status,Interest_Rate,Project_ID,Original_Principal_Amount,Cancelled_Amount,Undisbursed_Amount,Disbursed_Amount,Repaid_to_IBRD,Due_to_IBRD,Exchange_Adjustment,Borrowers_Obligation,Sold_3rd_Party,Repaid_3rd_Party,Due_3rd_Party,Loans_Held,First_Repayment_Date,Last_Repayment_Date,Agreement_Signing_Date,Board_Approval_Date,Effective_Date_Most_Recent,Closed_Date_Most_Recent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,786,699,7885,0,4059,430


After above steps, we wouldn't see any missing value except for the date related columns.
For date related columns, since they have more problems, we are not necessarily to remove all missing value. We have another way to deal with them.

There are 2 issues with date related columns:
1. Missing values
2. Having more than one date.
For each loan_number, we should only have one unique set of [First_Repayment_Date,Last_Repayment_Date,Agreement_Signing_Date,Board_Approval_Date,Effective_Date_Most_Recent,Closed_Date_Most_Recent],
However, we will see there are more than one set.

To handle this case, for each date related column, we can find the most frequent date of each loan number and use it as its only date. By doing this, we can find a unique set of date values for each loan number.
Therefore, we can create time table directly. Later we will need to create the transaction data, we can ignore the raw data of these date related columns and use loan number to recreate these columns.
This will also help us prevent spending time on cleaning missing data, so we can jump into data modeling directly.

### Step 4: Define the Data Model
#### Conceptual Data Model
In this project, I am using fact-dimensional data modeling method. The reason I am using this method is that it will be much more straightforward and easy for users.
We will have 1 fact table and 5 dimensional tables.

##### Fact table:
trainsaction:Loan_Number, Time_Id, Country_Id, Guarantor_Country_Id, Loan_Type, Loan_Status_Id, Amount_Id, End_of_Period, Interest_Rate, Project_ID, Exchange_Adjustment, Borrowers_Obligation, 
Cancelled_Amount,Undisbursed_Amount, Disbursed_Amount, Repaid_to_IBRD, Due_to_IBRD, Loans_Held

##### Dimension table

df_time: Time_Id, First_Repayment_Date, Last_Repayment_Date, Board_Approval_Date, Agreement_Signing_Date, Effective_Date_Most_Recent, Closed_Date_Most_Recent

df_country: Country_Id, Country_Code, Country, Region

df_amount: Amount_Id, Original_Principal_Amount, Sold_3rd_Party, Repaid_3rd_Party, Due_3rd_Party

df_loan_type: Loan_Type_Id, Loan_Type

df_loan_status: Loan_Status_Id, Loan_Status

In [ ]:
# Generate time dataframe 
# By exploring data, we can see that for each loan number, it should only have one set of dates related column. However, we will find for date related columns, it has more than one combination. so I crete this function to find its more frequent value for each loan number. 
def getss(df, column):
    # Create a sql table with loan number and column
    df.select('Loan_Number', column).filter('{} is not NULL'.format(column)).filter('{} != \'None\''.format(column)).createOrReplaceTempView("time")
    # Run query to get the most frequent value of each category for each loan number
    x = spark.sql("""
    SELECT Loan_Number, {} FROM 
    (
    SELECT Loan_Number, {}, count(1) total_records, ROW_NUMBER() OVER (PARTITION BY Loan_Number ORDER BY count(1) desc) AS seqnum
    FROM time 
    group by Loan_Number, {}
    )
    WHERE seqnum = 1
    """.format(column, column, column))
    return x
# Getting the most frequent value of each column for each loan number
x1 = getss(df, 'First_Repayment_Date')
x2 = getss(df, 'Last_Repayment_Date')
x3 = getss(df, 'Agreement_Signing_Date')
x4 = getss(df, 'Board_Approval_Date')
x5 = getss(df, 'Effective_Date_Most_Recent')
x6 = getss(df, 'Closed_Date_Most_Recent')
# Combine them and drop the duplicate key
df_time = x1.join(x2, x1.Loan_Number == x2.Loan_Number).drop(x2.Loan_Number)
df_time = df_time.join(x3, df_time.Loan_Number == x3.Loan_Number).drop(x3.Loan_Number)
df_time = df_time.join(x4, df_time.Loan_Number == x4.Loan_Number).drop(x4.Loan_Number)
df_time = df_time.join(x5, df_time.Loan_Number == x5.Loan_Number).drop(x5.Loan_Number)
df_time = df_time.join(x6, df_time.Loan_Number == x6.Loan_Number).drop(x6.Loan_Number)
# Add a unique id Time_Id for this dataframe
df_time = df_time.withColumn('Time_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
# Rename its column name
df_time = df_time.selectExpr('Time_Id', 'Loan_Number', 'First_Repayment_Date as First_Repayment_Date_t', 'Last_Repayment_Date as Last_Repayment_Date_t', 'Agreement_Signing_Date as Agreement_Signing_Date_t','Board_Approval_Date \
                         as Board_Approval_Date_t','Effective_Date_Most_Recent as Effective_Date_Most_Recent_t','Closed_Date_Most_Recent as Closed_Date_Most_Recent_t')

In [34]:
# Generate country dataframe 
df_country = df.select('Country_Code','Country','Region').distinct()
newRow = spark.createDataFrame([('GB','United Kingdom','EUROPE AND CENTRAL ASIA')])
df_country = df_country.union(newRow)
# Add a unique id Country_Id for this dataframe
df_country = df_country.withColumn('Country_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
# Casting to the right data type
fields = {'Country_Code':'string', 'Country':'string', 'Region':'string', 'Country_Id':'integer'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
df_country = df_country.selectExpr(*exprs)

In [36]:
# Generate Amount dataframe 
df_amount = df.select('Loan_Number','Original_Principal_Amount','Sold_3rd_Party','Repaid_3rd_Party', 'Due_3rd_Party').distinct()
# Add a unique id Amount_Id for this dataframe
df_amount = df_amount.withColumn('Amount_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [37]:
# Generate Loan_Type dataframe 
df_loan_type = df.select('Loan_Type').distinct()
# Add a unique id Loan_Type_Id for this dataframe
df_loan_type = df_loan_type.withColumn('Loan_Type_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
# Casting to the right data type
fields = {'Loan_Type':'string', 'Loan_Type_Id':'integer'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
df_loan_type = df_loan_type.selectExpr(*exprs)

In [38]:
# Generate Loan_Status dataframe 
df_loan_status = df.select('Loan_Status').distinct()
# Add a unique id Loan_Status_Id for this dataframe
df_loan_status = df_loan_status.withColumn('Loan_Status_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
# Casting to the right data type
fields = {'Loan_Status':'string', 'Loan_Status_Id':'integer'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
df_loan_status = df_loan_status.selectExpr(*exprs)

In [39]:
# create sql tables 
df_country.createOrReplaceTempView("country")
df_time.createOrReplaceTempView("time")
df_amount.createOrReplaceTempView("amount")
df_loan_type.createOrReplaceTempView("loan_type")
df_loan_status.createOrReplaceTempView("loan_status")
df.createOrReplaceTempView("log")

In [40]:
# Generate transaction dataframe by joing log table with the other tables on loan number
trainsaction = spark.sql("""
    select l.Loan_Number, t.Time_Id, c.Country_Id, cc.Country_Id as Guarantor_Country_Id, lt.Loan_Type, ls.Loan_Status_Id, a.Amount_Id,
    l.End_of_Period, l.Interest_Rate, l.Project_ID, l.Exchange_Adjustment, l.Borrowers_Obligation, l.Cancelled_Amount,
    l.Undisbursed_Amount, l.Disbursed_Amount, l.Repaid_to_IBRD, l.Due_to_IBRD, l.Loans_Held
    from log l 
    join country c on l.Country_Code= c.Country_Code
    join country cc on l.Guarantor = cc.Country 
    join loan_type lt on l.Loan_Type = lt.Loan_Type 
    join loan_status ls on l.Loan_Status = ls.Loan_Status
    join amount a on l.Loan_Number = a.Loan_Number
    join time t on l.Loan_Number = t.Loan_Number
    """)
# Casting to the right data type
fields = {'Loan_Number':'string', 'Time_Id':'integer', 'Country_Id':'integer', 'Guarantor_Country_Id':'integer',\
          'Loan_Type':'string','Loan_Status_Id':'integer', 'Amount_Id':'integer', 'End_of_Period':'timestamp', \
          'Interest_Rate':'float', 'Project_ID':'string', 'Exchange_Adjustment':'float','Borrowers_Obligation':'float',\
          'Cancelled_Amount':'float', 'Undisbursed_Amount':'float', 'Disbursed_Amount':'float','Repaid_to_IBRD':'float', 'Due_to_IBRD':'float', 'Loans_Held':'float'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
trainsaction = trainsaction.selectExpr(*exprs)

In [42]:
# drop the loan number column from the dataframes
df_amount_final = df_amount.select('Amount_Id','Original_Principal_Amount','Sold_3rd_Party','Repaid_3rd_Party', 'Due_3rd_Party')
# Casting to the right data type
fields = {'Amount_Id':'integer', 'Original_Principal_Amount':'float', 'Sold_3rd_Party':'float', 'Repaid_3rd_Party':'float', 'Due_3rd_Party':'float'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
df_amount_final = df_amount_final.selectExpr(*exprs)

In [43]:
# Rename the column names
df_time_final = df_time.selectExpr('Time_Id', 'First_Repayment_Date_t as First_Repayment_Date', \
                                   'Last_Repayment_Date_t as Last_Repayment_Date','Agreement_Signing_Date_t as Agreement_Signing_Date',\
                                   'Board_Approval_Date_t as Board_Approval_Date','Effective_Date_Most_Recent_t as Effective_Date_Most_Recent',\
                                   'Closed_Date_Most_Recent_t as Closed_Date_Most_Recent')
# Casting to the right data type
fields = {'Time_Id':'integer', 'First_Repayment_Date':'timestamp', 'Last_Repayment_Date':'timestamp', \
          'Agreement_Signing_Date':'timestamp', 'Board_Approval_Date':'timestamp', 'Effective_Date_Most_Recent':'timestamp', 'Closed_Date_Most_Recent':'timestamp'}
exprs = [ "cast ({} as {})".format(key,value) for key, value in fields.items()]
df_time_final = df_time_final.selectExpr(*exprs)

### Saving files to S3 
This process will take about 20 minutes. I already saved these files on AWS and you can ignore this part and go to next part directly.

In [ ]:
# Uncomment if you want to save data to AWS. The data
#start_time = time.time()
#df_country.write.csv("s3a://udacity-leejohn-w2/loan/country")
#df_time_final.write.csv("s3a://udacity-leejohn-w2/loan/time")
#df_loan_status.write.csv("s3a://udacity-leejohn-w2/loan/loan_status")
#df_loan_type.write.csv("s3a://udacity-leejohn-w2/loan/loan_type")
#df_amount_final.write.csv("s3a://udacity-leejohn-w2/loan/amount")
#trainsaction.write.csv("s3a://udacity-leejohn/loan-w2/trainsaction")
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Run this command to see the file on AWS
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )
## This is to display what we have in S3 
sampleDbBucket =  s3.Bucket("udacity-leejohn-w2")
i = 0
for obj in sampleDbBucket.objects.filter(Prefix="loan"):
    print(obj)

### Step 5: Run ETL to load the Data
First, run below script will create a redshift cluster and it will take about 5-10 min. 

Second, when you see "cluster is available", you can go ahread and run the etl.py.

In [ ]:
# This script will create a redshift cluster automatically, please don't move forward until you see "Cluster is available"
%run -i 'Redshift.py'

In [1]:
# This script will drop, create and load table into redshift cluster. You will see "All table dropped! All table created! All table loaded!" if everything works as exptected.
%run -i 'etl.py'

All table dropped!
All table created!
All table loaded!


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.